In [1]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
test = "mito_mem (3)"
print(test.split("(")[0].split(" ")[0])
for row in mi.rows:
    print(row.organelle_info)

mito_mem
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito_dna': 5, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito_dna': 5, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito_dna': 5, 'mito': 0}
{'mito_mem': 3, 'mito_lumen': 4, 'mito': 0}


In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
from annotator_metrics.src.preprocessing import copy_data
mi = MaskInformation()
print(mi.rows)
copy_data(group = "group1", output_base_path="/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/")

In [2]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
print(mi.rows[0].organelle_info)


{'mito_mem ': 3, 'mito_lumen ': 4, 'mito': 0}


In [3]:
import numpy as np
a=np.random.random((4,4,4))

In [7]:
print(a[1:3,1:3,1:3])

[[[0.29667459 0.14585225]
  [0.93038333 0.71463271]]

 [[0.04604659 0.92718152]
  [0.80627626 0.55996372]]]


In [2]:
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
results= calculate_all_to_all("group1","/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/", num_workers = 10)
print(results[1:10])

distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-5syvrpx1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-48jpuh33', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-vlpyi6pa', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-rh60rdp4', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-o1vjd11f', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/

http://127.0.0.1:8787/status
[Result(crop='01', organelle_name='mito_mem', metric_value='jaccard', metric_name='Jaccard Index', annotator_names=['b1', 'a2', 'gt', 'b2', 'c1', 'd1', 'e1', 'a1'], gt_idx=0, test_idx=0, sorting=-1, score=1.0), Result(crop='01', organelle_name='mito_mem', metric_value='hausdorff', metric_name='Hausdorff Distance', annotator_names=['b1', 'a2', 'gt', 'b2', 'c1', 'd1', 'e1', 'a1'], gt_idx=0, test_idx=0, sorting=1, score=0.0), Result(crop='01', organelle_name='mito_mem', metric_value='false_negative_rate', metric_name='False Negative Rate', annotator_names=['b1', 'a2', 'gt', 'b2', 'c1', 'd1', 'e1', 'a1'], gt_idx=0, test_idx=0, sorting=1, score=0.0), Result(crop='01', organelle_name='mito_mem', metric_value='false_negative_rate_with_tolerance', metric_name='False Negative Rate with Tolerance Distance', annotator_names=['b1', 'a2', 'gt', 'b2', 'c1', 'd1', 'e1', 'a1'], gt_idx=0, test_idx=0, sorting=1, score=0.0), Result(crop='01', organelle_name='mito_mem', metric

In [3]:
import numpy as np
all_to_all = {}
score_ranges = {}
for result in results:
    score_tuple = (result.crop,result.organelle_name,result.metric_value)
    if score_tuple not in all_to_all:
        all_to_all[score_tuple] = {
            "scores_matrix": np.zeros((len(result.annotator_names),len(result.annotator_names))),
            "annotator_names": result.annotator_names,
            "metric_name": result.metric_name
            }
    all_to_all[score_tuple]["scores_matrix"][result.gt_idx][result.test_idx] = result.score

    score_ranges_tuple = (result.organelle_name, result.metric_value)
    if score_ranges_tuple not in score_ranges:
        score_ranges[score_ranges_tuple]={
                        "min": np.nan_to_num(np.inf),
                        "max": np.nan_to_num(-np.inf),
                        "sorting": result.sorting,
                    }
    if result.gt_idx != result.test_idx:
        current_range = score_ranges[score_ranges_tuple]
        current_range["min"] = np.nanmin([current_range["min"], result.score])
        current_range["max"] = np.nanmax([current_range["max"], result.score])


In [9]:
from matplotlib import pyplot as plt
import matplotlib
import seaborn
import os
import dask
from dask import delayed
from dask.distributed import Client


def plot_figure(crop, organelle_name, metric_value, metric_name, annotator_names, all_to_all, score_range):
    matplotlib.rcParams["figure.dpi"] = 300
    matplotlib.rcParams['axes.facecolor']='white'
    matplotlib.rcParams['savefig.facecolor']='white'

    _, ax = plt.subplots(1, 1, figsize=(8, 6),)
    if score_range["sorting"] == 1:  # low to high
        sort_order = np.argsort(np.nanmean(all_to_all, axis=1))
        cmap = "rocket_r"
    else:
        cmap = "rocket"
        sort_order = np.argsort(np.nanmean(all_to_all, axis=1))[
            ::-1
        ]

    all_to_all = all_to_all[:, sort_order]
    all_to_all = all_to_all[sort_order, :]
    annotator_names = [annotator_names[s] for s in sort_order]
    seaborn.heatmap(
        all_to_all,
        annot=True,
        square=True,
        vmin=score_range["min"],
        vmax=score_range["max"],
        cmap=cmap,
    )
    if score_range["sorting"] == 1:  # low to high
        plt.gcf().axes[1].invert_yaxis()

    ax.set_title(organelle_name)
    ax.collections[0].colorbar.set_label(metric_name)
    ax.xaxis.tick_top()
    plt.xticks(
        [i + 0.5 for i in range(len(annotator_names))],
        annotator_names,
        rotation=45,
        ha="left",
    )
    plt.yticks(
        [i + 0.5 for i in range(len(annotator_names))],
        annotator_names,
        rotation=0,
    )
    output_directory = (
        f"{input_base_path}/plots/{group}/{crop}/{metric_value}"
    )

    os.makedirs(output_directory, exist_ok=True)
    try:
        os.remove(f"{output_directory}/{organelle_name}.png")
    except OSError:
        pass
    plt.savefig(
        f"{output_directory}/{organelle_name}.png", bbox_inches="tight"
    )
    plt.close()

client = Client(n_workers=10, threads_per_worker=1)
print(client.dashboard_link)

group = "group01"
input_base_path = '/groups/cosem/cosem/ackermand/annotation_and_analytics/'
lazy_results = []
for score_tuple, score_entry in all_to_all.items():
    crop = score_tuple[0]
    organelle_name = score_tuple[1]
    metric_value = score_tuple[2]
    metric_name = score_entry["metric_name"]
    annotator_names = score_entry["annotator_names"]
    current_all_to_all = score_entry["scores_matrix"]
    score_range = score_ranges[(organelle_name, metric_value)]
    lazy_results.append(delayed(plot_figure)(crop, organelle_name, metric_value, metric_name, annotator_names, current_all_to_all, score_range))
    
dask.compute(*lazy_results)

/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38623 instead
  f"Port {expected} is already in use.\n"


http://127.0.0.1:38623/status


ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty slice
ipykernel_launcher:21: RuntimeWarning: Mean of empty slice
ipykernel_launcher:17: RuntimeWarning: Mean of empty sli

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [6]:
import dask
import dask.distributed
dask.config.set({"distributed.comm.timeouts.connect":100})
dask.config.get("distributed.comm.timeouts.connect")

100